In [1]:
import findspark

findspark.init()
findspark.find()

'C:\\Users\\mehrmall\\spark'

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Sale Analysis').getOrCreate()
spark

In [71]:
data = spark.read.csv('Train.csv', header=True, inferSchema=True)
data.show(8)

+----+-------+----+------------+--------------------+------+------------------+-------------------+-------------------+
|  ID|CONSOLE|YEAR|    CATEGORY|           PUBLISHER|RATING|    CRITICS_POINTS|        USER_POINTS|    SalesInMillions|
+----+-------+----+------------+--------------------+------+------------------+-------------------+-------------------+
|2860|     ds|2008|role-playing|            Nintendo|     E|2.8333333333333335| 0.3037037037037037| 1.7792573645377137|
| 731|    wii|2012|  simulation|Konami Digital En...|  E10+|              13.2|               1.64|0.21504956996959468|
| 495|     pc|2019|     shooter|          Activision|     M|            4.5625|0.00641025641025641| 0.5344016462886677|
|2641|    ps2|2002|      sports|     Electronic Arts|     E| 4.181818181818182| 0.3269230769230769| 1.3839642351062431|
| 811|    ps3|2013|      action|          Activision|     M| 2.259259259259259|0.03257918552036199|0.08267053460319962|
|1803|    ps2|2004|    platform|Sony Com

In [72]:
data = data.drop('ID')

In [ ]:
from pyspark.sql.functions import isnull, isnan, count, when, col, countDistinct

In [19]:
data.select([count(when((isnan(c)|isnull(c)), c)).alias(c) for c in data.columns]).show()

+---+-------+----+--------+---------+------+--------------+-----------+---------------+
| ID|CONSOLE|YEAR|CATEGORY|PUBLISHER|RATING|CRITICS_POINTS|USER_POINTS|SalesInMillions|
+---+-------+----+--------+---------+------+--------------+-----------+---------------+
|  0|      0|   0|       0|        0|     0|             0|          0|              0|
+---+-------+----+--------+---------+------+--------------+-----------+---------------+



In [73]:
data.printSchema()

root
 |-- CONSOLE: string (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- PUBLISHER: string (nullable = true)
 |-- RATING: string (nullable = true)
 |-- CRITICS_POINTS: double (nullable = true)
 |-- USER_POINTS: double (nullable = true)
 |-- SalesInMillions: double (nullable = true)



In [31]:
data.agg(*(countDistinct(col(c)).alias(c) for c in data.columns)).show()

+-------+----+--------+---------+------+--------------+-----------+---------------+
|CONSOLE|YEAR|CATEGORY|PUBLISHER|RATING|CRITICS_POINTS|USER_POINTS|SalesInMillions|
+-------+----+--------+---------+------+--------------+-----------+---------------+
|     17|  23|      12|      204|     6|          1683|       2202|           3506|
+-------+----+--------+---------+------+--------------+-----------+---------------+



In [77]:
data.groupBy('CONSOLE').sum('SalesInMillions').show()

+-------+--------------------+
|CONSOLE|sum(SalesInMillions)|
+-------+--------------------+
|    ps3|   986.0796071263729|
|    psv|  105.85129867090379|
|   x360|   1086.224411747302|
|      x|   462.3871834483602|
|     ps|  197.34687742096602|
|    psp|  337.44769535175175|
|    ps2|  1273.7950645303797|
|     ds|   526.7266360224263|
|   xone|   185.8433670999068|
|    wii|   743.1045899652238|
|     dc|   8.765281251187663|
|    3ds|  178.42191798919606|
|   wiiu|  101.45957723819255|
|     gc|   310.3876118753533|
|     pc|   549.5314770515059|
|    gba|  255.90303048545456|
|    ps4|  302.32348396169664|
+-------+--------------------+



In [35]:
data.groupBy('RATING').avg('USER_POINTS').show()

+------+-------------------+
|RATING|   avg(USER_POINTS)|
+------+-------------------+
|     E| 0.5510362333296336|
|     T|0.39674710444301803|
|   K-A|0.15416666666666667|
|     M|0.17590941618530223|
|    RP| 0.5059259259259259|
|  E10+| 0.4624107654743019|
+------+-------------------+



In [43]:
data.groupBy('PUBLISHER').sum('SalesInMillions').orderBy('sum(SalesInMillions)', ascending=False).show()

+--------------------+--------------------+
|           PUBLISHER|sum(SalesInMillions)|
+--------------------+--------------------+
|     Electronic Arts|  1145.5190156365447|
|            Nintendo|   786.3247579236017|
|          Activision|   678.2391367369747|
|             Ubisoft|  497.37845537188645|
|Sony Computer Ent...|   383.3700066355024|
|Take-Two Interactive|  352.08990325645334|
|                 Thq|  341.95171371031245|
|                Sega|  268.30487675096987|
|Microsoft Game St...|   255.1983601608654|
|Konami Digital En...|   195.1477716192783|
|  Namco Bandai Games|  192.49556183569206|
|               Atari|   169.7820858473409|
|              Capcom|   169.6484144273985|
|Warner Bros. Inte...|  152.74979641552588|
|         Square Enix|  141.62153790852403|
|          Tecmo Koei|  125.46722388417533|
|        Midway Games|  105.79848721898134|
|       Vivendi Games|   97.04919206101394|
|   Eidos Interactive|    92.0055968718395|
|  Bethesda Softworks|   90.8788

In [46]:
data.groupBy("CATEGORY").sum("SalesInMillions").show()

+------------+--------------------+
|    CATEGORY|sum(SalesInMillions)|
+------------+--------------------+
|     shooter|  1085.6353916537596|
|      action|   1731.855429148815|
|    strategy|    206.302355121276|
|    fighting|   363.9712335448475|
|      racing|   672.6629381963066|
|   adventure|   216.8839519102497|
|        misc|   521.2175735745843|
|  simulation|   308.9161357776841|
|      sports|  1092.8326375030263|
|role-playing|   794.9792650004375|
|    platform|  506.64053683553317|
|      puzzle|  109.70166296966089|
+------------+--------------------+



In [53]:
games_by_cat = data.groupBy("CATEGORY").count()
total_games = data.count()
games_by_cat.withColumn("% of Games", games_by_cat["count"] / total_games * 100).show()

+------------+-----+------------------+
|    CATEGORY|count|        % of Games|
+------------+-----+------------------+
|     shooter|  441| 12.57843696520251|
|      action|  845| 24.10154021677125|
|    strategy|  132|3.7649743297204794|
|    fighting|  187| 5.333713633770679|
|      racing|  296| 8.442669709070165|
|   adventure|  114|3.2515687393040507|
|        misc|  220| 6.274957216200798|
|  simulation|  150| 4.278379920136908|
|      sports|  464|13.234455219623504|
|role-playing|  380|10.838562464346833|
|    platform|  218| 6.217912150598973|
|      puzzle|   59|1.6828294352538506|
+------------+-----+------------------+



In [30]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

In [31]:
cat_cols = ["CONSOLE", "CATEGORY", "PUBLISHER", "RATING"]
stages = []

for c in cat_cols:
    stringIndexer = StringIndexer(inputCol=c, outputCol=c + "_index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], \
            outputCols=[c + "_vec"])    
    stages += [stringIndexer, encoder]
    
num_cols = ["YEAR", "CRITICS_POINTS", "USER_POINTS"]
assemblerInputs = [c + "_vec" for c in cat_cols] + num_cols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

pipeline = Pipeline(stages=stages)
pipeline = pipeline.fit(data)
train = pipeline.transform(data)

In [32]:
scaler = StandardScaler().setInputCol('features').setOutputCol('scaled_features')
scaler = scaler.fit(train)
scaler

StandardScalerModel: uid=StandardScaler_366e3b0ed61d, numFeatures=238, withMean=false, withStd=true

In [33]:
train = scaler.transform(train)

In [51]:
rfl = RandomForestRegressor(featuresCol='scaled_features', labelCol='SalesInMillions', numTrees=80)
rfl = rfl.fit(train)
rfl

RandomForestRegressionModel: uid=RandomForestRegressor_a363160aad5d, numTrees=80, numFeatures=238

In [66]:
test = spark.read.csv('Test.csv', header=True, inferSchema=True)
test = test.drop('ID')
test.show(5)

+-------+----+--------+---------------+------+------------------+-------------------+
|CONSOLE|YEAR|CATEGORY|      PUBLISHER|RATING|    CRITICS_POINTS|        USER_POINTS|
+-------+----+--------+---------------+------+------------------+-------------------+
|    ps2|2008|  action|     Tecmo Koei|     T|3.9285714285714284|0.48235294117647054|
|    psp|2007|strategy|          Atari|  E10+| 5.538461538461538|0.07179487179487179|
|    ps2|2004| shooter|Electronic Arts|     T|3.0344827586206895|0.06204379562043796|
|      x|2006|  action|Electronic Arts|     E|2.9130434782608696| 0.8800000000000001|
|    3ds|2011|  racing|        Ubisoft|  E10+| 1.162162162162162|0.18333333333333335|
+-------+----+--------+---------------+------+------------------+-------------------+
only showing top 5 rows



In [36]:
target = spark.read.csv("Sample_Submission.csv", header=True, inferSchema=True)
target.show(3)

+------------------+
|   SalesInMillions|
+------------------+
|1.4975327784054218|
| 2.368795198397184|
|2.6305046920917925|
+------------------+
only showing top 3 rows



In [37]:
test = test.join(target)

In [38]:
test = pipeline.transform(test)
test = scaler.transform(test)

In [52]:
test_pred = rfl.transform(test)

In [53]:
train_pred = rfl.transform(train)

In [61]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol='SalesInMillions', metricName="rmse")

print("TRAIN RMSE:", evaluator.evaluate(train_pred))

TRAIN RMSE: 1.68875269317499


In [69]:
spark.read.csv?